In [260]:
print('hello IE 507_Lab 3')

hello IE 507_Lab 3


In [261]:
!pip install -q pyomo

Exercise 2

The optimization problem is

max 2x1 + 3x2 + 5x3 + 4x4 + 6x5 + 6x6 + 8x7 + 8x8 + 6x9 + 4x10 + 6x11

s.t.

-2x1 - 3x2 - 5x3 - 4x4 - 6x5 - 6x6 - 8x7 - 8x8 - 6x9 - 4x10 - 6x11 <= -26

-x1 - x2 - x3 - x4 - x5 - x6 - x7 - x8 - x9 - x10 - x11 <= -5

x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 <=9

x6 + x8 <=1

x4 + x10 <=1

x3 + x8 <=1

-2x1 + x11 <=0

x1 + x2 <=1

-x1 - x1 <=-1

x3 + x5 <=1

x9 + x11 <=1

In [262]:
from pyomo.environ import*

In [263]:
import numpy as np

In [264]:
model = ConcreteModel()

In [265]:
N = 11
M = 11 
lb=np.array([0,0,0,0,0,0,0,0,0,0,0])
ub=np.array([1,1,1,1,1,1,1,1,1,1,1])
obj_coef_c = np.array([2,3,5,4,6,6,8,8,6,4,6]) 
constr_coef_A=np.array([[-2,-3,-5,-4,-6,-6,-8,-8,-6,-4,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1,1],[0,0,0,0,0,1,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,1,0],[0,0,1,0,0,0,0,1,0,0,0],[-2,0,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,0,0],[-1,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,1 ],[0,0,1,0,1,0,0,0,0,0,0]])
constr_rhs_b=np.array([-26,-5,9,1,1,1,0,1,-1,1,1])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [266]:
model.x=Var(col_indices,domain=Binary)
model.constraints = ConstraintList()

In [267]:
for i in col_indices:
  model.x[i].setlb(lb[i])
  model.x[i].setub(ub[i])

In [268]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [269]:
model.objective = Objective(expr =summation(obj_coef_c,model.x), sense=maximize)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None 

In [270]:
!apt-get install -y -qq coinor-cbc

In [271]:
opt_cbc = SolverFactory('cbc')

In [272]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 35.0
  Upper bound: 35.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03205299377441406
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [273]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraints.display()


Objective =  35.0

Decision Variables
x[ 0 ] =  0.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[ 10 ] =  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -35.0 : -26.0
      2 :  None :  -6.0 :  -5.0
      3 :  None :   6.0 :   9.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0


3.[R]Solver your MILP usingcbcsolver and report the optimal values of the variables and theobjective function value

 The objective function value is 

Objective =  35.0

optimal values of the variable is 

Decision Variables
x[ 0 ] =  0.0       
x[ 1 ] =  1.0       
x[ 2 ] =  0.0         
x[ 3 ] =  0.0      
x[ 4 ] =  1.0      
x[ 5 ] =  0.0      
x[ 6 ] =  1.0      
x[ 7 ] =  1.0       
x[ 8 ] =  1.0      
x[ 9 ] =  1.0    
x[ 10 ] =  0.0       
 
 The constraints acyivity is 

 Constraints          
constraints : Size=11              
    Key : Lower : Body  : Upper                    
      1 :  None : -35.0 : -26.0                      
      2 :  None :  -6.0 :  -5.0                       
      3 :  None :   6.0 :   9.0                      
      4 :  None :   1.0 :   1.0                       
      5 :  None :   1.0 :   1.0                       
      6 :  None :   1.0 :   1.0              
      7 :  None :   0.0 :   0.0                      
      8 :  None :   1.0 :   1.0               
      9 :  None :  -1.0 :  -1.0                     
     10 :  None :   1.0 :   1.0                       
     11 :  None :   1.0 :   1.0                      


4.[R]Let  us  now  compare  it  to  a  linear  program.   Suppose  we  remove  the  restrictions  thatthe variables in the above problem are integers.  Solve this modified problem and report theoptimal values of the variables and the objective function value

In [274]:
!pip install -q pyomo

In [275]:
from pyomo.environ import*

In [276]:
import numpy as np

In [277]:
model=ConcreteModel()

In [278]:
N = 11
M = 11 
lb=np.array([0,0,0,0,0,0,0,0,0,0,0])
ub=np.array([1,1,1,1,1,1,1,1,1,1,1])
obj_coef_c = np.array([2,3,5,4,6,6,8,8,6,4,6]) 
constr_coef_A=np.array([[-2,-3,-5,-4,-6,-6,-8,-8,-6,-4,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1,1],[0,0,0,0,0,1,0,1,0,0,0],[0,0,0,1,0,0,0,0,0,1,0],[0,0,1,0,0,0,0,1,0,0,0],[-2,0,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,0,0],[-1,-1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,0,1 ],[0,0,1,0,1,0,0,0,0,0,0]])
constr_rhs_b=np.array([-26,-5,9,1,1,1,0,1,-1,1,1])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [279]:
model.x=Var(col_indices,domain=Reals)
model.constraints = ConstraintList()

In [280]:
for i in col_indices:
  model.x[i].setlb(lb[i])
  model.x[i].setub(ub[i])

In [281]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [282]:
model.objective = Objective(expr =summation(obj_coef_c,model.x), sense=maximize)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None 

In [283]:
!apt-get install -y -qq coinor-cbc

In [284]:
opt_cbc = SolverFactory('cbc')

In [285]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 35.0
  Upper bound: 35.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 5
  Error rc: 0
  Time: 0.022014379501342773
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


6.[R]Suppose  a  new  courseC11with  9  credits  is  introduced,  but  the  course  timing  forC11overlaps with the course timings ofC7, C8, how will you change your model to include thisnew constraint?

Now  a new decision variable x12 has to be added to the problem and we need to modify the objective function as 

max 2x1 + 3x2 + 5x3 + 4x4 + 6x5 + 6x6 + 8x7 + 8x8 + 6x9 + 4x10 + 6x11 + 9x12

 and add  new constraints x8 +x12 <=1 and x9 +x12<=1
 
  We can do the following

In [287]:
N = 12
model.x12 = Var() 

In [288]:
#to add the new constraint we do the following
model.constraints.add(expr = model.x[7]+ model.x12 <= 1)
model.constraints.add(expr = model.x[8]+ model.x12 <= 1)


In [289]:
model.new_objective = Objective(expr = 9*model.x12 + summation(obj_coef_c,model.x),sense=maximize )


In [290]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   13 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

2 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False :  Reals
          1 :     0 :   1.0 :     1 : False : False :  Reals
          2 :     0 :   0.0 :     1 : False : False :  Reals
          3 :     0 :   0.0 :     1 : False : False :  Reals
          4 :     0 :   1.0 :     1 : False : False :  Reals
          5 :     0 :   0.0 :     1 : False : False :  Reals
          6 :     0 :   1.0 :     1 : False : False :  Reals
          7 :     0 :   1.0 :     1 : False : False :  Reals
          8 :     0 

In [291]:
model.objective.deactivate()

In [292]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 41.5
  Upper bound: 41.5
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 13
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.01890110969543457
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [293]:
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraints.display()


Objective =  41.5

Decision Variables
x[ 0 ] =  0.5
x[ 1 ] =  0.5
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0

Constraints
constraints : Size=13
    Key : Lower : Body  : Upper
      1 :  None : -32.5 : -26.0
      2 :  None :  -6.0 :  -5.0
      3 :  None :   6.0 :   9.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   0.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   1.0 :   1.0
      9 :  None :  -1.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   1.0 :   1.0
     12 :  None :   1.0 :   1.0
     13 :  None :   1.0 :   1.0
